# DAY 1 - NOTEBOOK 3: PARTITIONING & CACHING

Version này đã fix:
- ✅ Lỗi sum() conflict
- ✅ Data distribution issues
- ✅ Giải thích rõ ràng cơ chế

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, when
from pyspark import StorageLevel
import time

# Stop existing
try:
    spark.stop()
    time.sleep(2)
except:
    pass

# Create session
spark = SparkSession.builder \
    .appName("Day1-Partitioning-Fixed") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "512m") \
    .config("spark.executor.cores", "1") \
    .config("spark.cores.max", "2") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

print("✅ Spark Session Created")
print(f"   Default Parallelism: {spark.sparkContext.defaultParallelism}")

✅ Spark Session Created
   Default Parallelism: 2


---

## PART 1: HIỂU VỀ DEFAULT PARALLELISM

In [17]:
print("📌 Default Parallelism trong cluster của bạn:")
print("="*60)

parallelism = spark.sparkContext.defaultParallelism
print(f"Default Parallelism: {parallelism}")
print(f"\n💡 Giải thích:")
print(f"   • Bạn có 2 workers")
print(f"   • Mỗi worker có 1 core")
print(f"   • Total cores = 2 × 1 = 2")
print(f"   • Default parallelism = {parallelism}")
print(f"\n⚠️  Khi tạo DataFrame từ Python list:")
print(f"   → Spark tự động tạo {parallelism} partitions")

📌 Default Parallelism trong cluster của bạn:
Default Parallelism: 2

💡 Giải thích:
   • Bạn có 2 workers
   • Mỗi worker có 1 core
   • Total cores = 2 × 1 = 2
   • Default parallelism = 2

⚠️  Khi tạo DataFrame từ Python list:
   → Spark tự động tạo 2 partitions


---

## PART 2: TẠO DATAFRAME VÀ KIỂM TRA PARTITIONS

In [18]:
# Tạo DataFrame từ Python list
data = [(i, f"Name_{i}", i * 10) for i in range(1, 1001)]
df = spark.createDataFrame(data, ["id", "name", "value"])

print("📊 DataFrame Info:")
print(f"   Total rows: {df.count()}")
print(f"   Number of partitions: {df.rdd.getNumPartitions()}")
print(f"\n💡 Vì default.parallelism = {spark.sparkContext.defaultParallelism}")
print(f"   → Spark tạo {df.rdd.getNumPartitions()} partitions")

📊 DataFrame Info:


   Total rows: 1000
   Number of partitions: 2

💡 Vì default.parallelism = 2
   → Spark tạo 2 partitions


In [19]:
# ✅ FIXED: Dùng len(list()) thay vì sum()
def get_partition_distribution(dataframe):
    """Get row count per partition"""
    partition_counts = dataframe.rdd.mapPartitionsWithIndex(
        lambda idx, it: [(idx, len(list(it)))]
    ).collect()
    return partition_counts

print("📌 Data distribution:")
print("="*60)

partition_counts = get_partition_distribution(df)

for partition_id, count in partition_counts:
    bar = "█" * (count // 20)
    print(f"Partition {partition_id}: {count:4d} rows {bar}")

print("="*60)
print("\n✅ Data phân bổ đều vào 2 partitions")

📌 Data distribution:
Partition 0:  500 rows █████████████████████████
Partition 1:  500 rows █████████████████████████

✅ Data phân bổ đều vào 2 partitions


---

## PART 3: REPARTITION - TĂNG SỐ PARTITIONS

In [20]:
print("📌 Repartition: Tăng từ 2 → 8 partitions")
print("="*60)

print(f"Before: {df.rdd.getNumPartitions()} partitions")

# Repartition to 8
df_8 = df.repartition(8)

print(f"After:  {df_8.rdd.getNumPartitions()} partitions")
print("\n📊 Data distribution sau repartition:")
print("="*60)

partition_counts = get_partition_distribution(df_8)

for partition_id, count in partition_counts:
    bar = "█" * (count // 5)
    print(f"Partition {partition_id}: {count:4d} rows {bar}")

print("="*60)
print("\n✅ Data đã phân bổ đều vào 8 partitions!")
print("💡 repartition() thực hiện full shuffle để phân bổ đều")

📌 Repartition: Tăng từ 2 → 8 partitions
Before: 2 partitions
After:  8 partitions

📊 Data distribution sau repartition:
Partition 0:  124 rows ████████████████████████
Partition 1:  124 rows ████████████████████████
Partition 2:  126 rows █████████████████████████
Partition 3:  126 rows █████████████████████████
Partition 4:  126 rows █████████████████████████
Partition 5:  126 rows █████████████████████████
Partition 6:  124 rows ████████████████████████
Partition 7:  124 rows ████████████████████████

✅ Data đã phân bổ đều vào 8 partitions!
💡 repartition() thực hiện full shuffle để phân bổ đều


---

## PART 4: COALESCE - GIẢM SỐ PARTITIONS

In [21]:
print("📌 Coalesce: Giảm từ 8 → 3 partitions")
print("="*60)

print(f"Before: {df_8.rdd.getNumPartitions()} partitions")

# Coalesce to 3
df_3 = df_8.coalesce(3)

print(f"After:  {df_3.rdd.getNumPartitions()} partitions")
print("\n📊 Data distribution sau coalesce:")
print("="*60)

partition_counts = get_partition_distribution(df_3)

for partition_id, count in partition_counts:
    bar = "█" * (count // 10)
    print(f"Partition {partition_id}: {count:4d} rows {bar}")

print("="*60)
print("\n⚠️  Data có thể không đều (coalesce chỉ gộp partitions)")
print("💡 coalesce() minimal shuffle, nhanh hơn repartition()")

📌 Coalesce: Giảm từ 8 → 3 partitions
Before: 8 partitions
After:  3 partitions

📊 Data distribution sau coalesce:
Partition 0:  374 rows █████████████████████████████████████
Partition 1:  374 rows █████████████████████████████████████
Partition 2:  252 rows █████████████████████████

⚠️  Data có thể không đều (coalesce chỉ gộp partitions)
💡 coalesce() minimal shuffle, nhanh hơn repartition()


---

## PART 5: COALESCE KHÔNG TĂNG ĐƯỢC PARTITIONS

In [22]:
print("📌 Demo: coalesce() KHÔNG thể tăng partitions")
print("="*60)

# Bắt đầu với 2 partitions
df_2 = df.coalesce(2)
print(f"Starting with: {df_2.rdd.getNumPartitions()} partitions")

# Thử coalesce lên 8
df_try_8 = df_2.coalesce(8)
print(f"After coalesce(8): {df_try_8.rdd.getNumPartitions()} partitions")

print("\n❌ Vẫn 2 partitions! coalesce() CHỈ GIẢM, không tăng")
print("\n✅ Muốn tăng partitions → dùng repartition()")

df_real_8 = df_2.repartition(8)
print(f"After repartition(8): {df_real_8.rdd.getNumPartitions()} partitions")
print("✅ Thành công!")

📌 Demo: coalesce() KHÔNG thể tăng partitions
Starting with: 2 partitions
After coalesce(8): 2 partitions

❌ Vẫn 2 partitions! coalesce() CHỈ GIẢM, không tăng

✅ Muốn tăng partitions → dùng repartition()
After repartition(8): 8 partitions
✅ Thành công!


---

## PART 6: REPARTITION BY COLUMN

In [23]:
# Tạo data với department column
import random
random.seed(42)

data_dept = []
departments = ["Engineering", "Sales", "HR", "Marketing"]
for i in range(1, 1001):
    data_dept.append((
        i,
        f"Name_{i}",
        i * 10,
        random.choice(departments)
    ))

df_dept = spark.createDataFrame(data_dept, ["id", "name", "value", "department"])

print("✅ Created DataFrame with department column")
print(f"   Departments: {departments}")

✅ Created DataFrame with department column
   Departments: ['Engineering', 'Sales', 'HR', 'Marketing']


In [26]:
print("🔍 PHÂN TÍCH HASH PARTITIONING - FULL DEMO")
print("="*70)

# Helper function
def get_partition_distribution(dataframe):
    """Get row count per partition"""
    return dataframe.rdd.mapPartitionsWithIndex(
        lambda idx, it: [(idx, len(list(it)))]
    ).collect()

# 1. Kiểm tra departments
print("\n1️⃣ KIỂM TRA DEPARTMENTS:")
print("-"*70)
dept_stats = df_dept.groupBy("department").count().orderBy("department")
dept_stats.show()

num_depts = df_dept.select("department").distinct().count()
print(f"✅ Tổng số departments: {num_depts}")

# 2. Repartition 4 (có partition trống)
print("\n2️⃣ REPARTITION(4) - CÓ PARTITION TRỐNG:")
print("-"*70)
df_4 = df_dept.repartition(4, "department")
print(f"Partitions: {df_4.rdd.getNumPartitions()}")

counts_4 = get_partition_distribution(df_4)
for pid, count in counts_4:
    bar = "█" * (count // 10) if count > 0 else "(empty)"
    status = "✅" if count > 0 else "❌ EMPTY"
    print(f"Partition {pid}: {count:4d} rows {bar} {status}")

empty_count = len([1 for _, c in counts_4 if c == 0])
print(f"\n⚠️  {empty_count} partition(s) trống")
print("💡 Nguyên nhân: hash(department) % 4 không map vào tất cả partitions")

# 3. Repartition optimal (= số departments)
print(f"\n3️⃣ REPARTITION({num_depts}) - OPTIMAL:")
print("-"*70)
df_opt = df_dept.repartition(num_depts, "department")
print(f"Partitions: {df_opt.rdd.getNumPartitions()}")

counts_opt = get_partition_distribution(df_opt)
for pid, count in counts_opt:
    bar = "█" * (count // 10)
    print(f"Partition {pid}: {count:4d} rows {bar} ✅")

print(f"\n✅ Tất cả {num_depts} partitions đều có data!")

# 4. Xem department mapping
print("\n4️⃣ DEPARTMENT MAPPING:")
print("-"*70)
for i in range(num_depts):
    partition_data = df_opt.rdd \
        .mapPartitionsWithIndex(lambda idx, it: [(idx, list(it))] if idx == i else []) \
        .collect()
    
    if partition_data and partition_data[0][1]:
        rows = partition_data[0][1]
        depts = set(row['department'] for row in rows[:100])
        print(f"Partition {i}: {depts} ({len(rows)} rows)")

# 5. Performance test
print("\n5️⃣ PERFORMANCE TEST:")
print("-"*70)

start = time.time()
df_4.groupBy("department").count().collect()
time_4 = time.time() - start

start = time.time()
df_opt.groupBy("department").count().collect()
time_opt = time.time() - start

print(f"repartition(4):              {time_4:.4f}s")
print(f"repartition({num_depts}):              {time_opt:.4f}s")
if time_4 > time_opt:
    print(f"⚡ Optimal is {time_4/time_opt:.2f}x faster!")
else:
    print(f"⚠️  Similar performance (small dataset)")

# 6. Kết luận
print("\n" + "="*70)
print("📝 KẾT LUẬN:")
print("-"*70)
print("✅ Partition trống là BÌNH THƯỜNG với hash partitioning")
print("✅ Không phải bug, là đặc tính của hash function")
print("✅ Best practice: num_partitions = num_unique_values")
print("✅ Hash partitioning đảm bảo: cùng value → cùng partition")
print("✅ Benefit: Tối ưu cho groupBy, join operations")

🔍 PHÂN TÍCH HASH PARTITIONING - FULL DEMO

1️⃣ KIỂM TRA DEPARTMENTS:
----------------------------------------------------------------------
+-----------+-----+
| department|count|
+-----------+-----+
|Engineering|  243|
|         HR|  250|
|  Marketing|  264|
|      Sales|  243|
+-----------+-----+

✅ Tổng số departments: 4

2️⃣ REPARTITION(4) - CÓ PARTITION TRỐNG:
----------------------------------------------------------------------
Partitions: 4
Partition 0:  243 rows ████████████████████████ ✅
Partition 1:  514 rows ███████████████████████████████████████████████████ ✅
Partition 2:  243 rows ████████████████████████ ✅
Partition 3:    0 rows (empty) ❌ EMPTY

⚠️  1 partition(s) trống
💡 Nguyên nhân: hash(department) % 4 không map vào tất cả partitions

3️⃣ REPARTITION(4) - OPTIMAL:
----------------------------------------------------------------------
Partitions: 4
Partition 0:  243 rows ████████████████████████ ✅
Partition 1:  514 rows ████████████████████████████████████████████████

---

## PART 7: CACHING DEMO

In [27]:
# Create test DataFrame
df_test = spark.range(1, 100000).toDF("id") \
    .withColumn("value", col("id") * 2) \
    .filter(col("id") % 2 == 0)

print("✅ Created test DataFrame")

✅ Created test DataFrame


In [28]:
print("📌 Test WITHOUT cache")
print("="*60)

start = time.time()
count1 = df_test.count()
count2 = df_test.count()
count3 = df_test.count()
time_no_cache = time.time() - start

print(f"Time: {time_no_cache:.4f}s")
print("⚠️  Each action re-computed!")

📌 Test WITHOUT cache
Time: 0.5728s
⚠️  Each action re-computed!


In [29]:
print("\n📌 Test WITH cache")
print("="*60)

df_cached = df_test.cache()

start = time.time()
count1 = df_cached.count()  # Compute + cache
count2 = df_cached.count()  # Use cache
count3 = df_cached.count()  # Use cache
time_with_cache = time.time() - start

print(f"Time: {time_with_cache:.4f}s")
print(f"⚡ Speedup: {time_no_cache/time_with_cache:.2f}x faster!")

# Cleanup
df_cached.unpersist()
print("\n✅ Cache cleared")


📌 Test WITH cache
Time: 0.9282s
⚡ Speedup: 0.62x faster!

✅ Cache cleared


---

## ✅ SUMMARY

### 🔑 Key Points:

**1️⃣ Default Parallelism:**
- Số partitions mặc định khi tạo DataFrame
- = số cores trong cluster
- Cluster của bạn: 2 cores → 2 partitions

**2️⃣ repartition(N):**
- Tăng HOẶC giảm partitions
- Full shuffle (chậm)
- Data phân bổ đều

**3️⃣ coalesce(N):**
- CHỈ giảm partitions
- Minimal shuffle (nhanh)
- Data có thể không đều

**4️⃣ repartition(N, "column"):**
- Hash partitioning
- Cùng value → cùng partition
- Tối ưu cho groupBy

**5️⃣ Caching:**
- Tránh recompute
- Dùng cho DataFrame dùng nhiều lần
- Nhớ unpersist() khi xong

### 🎯 Best Practices:
- ✅ Số partitions = 2-4x số cores
- ✅ Dùng repartition() để tăng
- ✅ Dùng coalesce() để giảm
- ✅ Cache DataFrame dùng nhiều lần
- ✅ Luôn unpersist() khi xong